# Computer Vision Based EKG Understanding
Originally developed for MIT 6.8300 Spring 2023 final project.

# Basics

In [ ]:
####!!!!!!!!!!!!!! NEED TO ADDRESS DATA AUGMENETATION
train_the_model = False
#model_label = "STTC_MI_NORM_HYP_CD_fullsize"
#model_label = "STTC_MI_NORM_HYP_CD_fullsize_1000"
#model_label = "STTC_MI_NORM_HYP_CD"
model_label = "NORM_1AVB_WPW_TRIGU_alternate"
#image_dir = 'drive/MyDrive/68300/MLEKG/image_dir_full_reduced_4/'
#image_dir = 'drive/MyDrive/68300/MLEKG/image_dir_full_reduced_2/'


#image_dir = 'drive/MyDrive/68300/MLEKG/figures_tiny/'
#image_dir = 'drive/MyDrive/68300/MLEKG/figures_99/'
#image_dir = 'drive/MyDrive/68300/MLEKG/figures_test_batch/'


#image_dir = 'drive/MyDrive/68300/MLEKG/image_dir_reduced_2'

image_dir = 'drive/MyDrive/68300/MLEKG/image_dir_full_reduced_2_fullsize/'
#image_dir = 'drive/MyDrive/68300/MLEKG/20000/full_1000_2/'

#image_dir = 'drive/MyDrive/68300/MLEKG/Figs_3000/'
#image_dir = 'drive/MyDrive/68300/MLEKG/figures/'

labels_csv = 'drive/MyDrive/68300/MLEKG/ptbxl_database.csv'
scp_statements_csv = 'drive/MyDrive/68300/MLEKG/scp_statements_mod.csv'

image_name_suffix = "_hr.png"


model_name = 'vgg' 
batch_size = 2  #16
num_epochs = 5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
save_dir = 'drive/MyDrive/68300/MLEKG/trained_models/{}'.format(model_label)
#classes_to_include = ['MI','STTC','NORM','CD','HYP']
classes_to_include = ['NORM','1AVB','WPW','TRIGU']




In [ ]:
import os
import ast
import pandas as pd
from torch.utils.data import WeightedRandomSampler
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
!pip install tqdm
from tqdm.notebook import tqdm
import os
import copy
import pandas as pd
import PIL 
import random
from collections import defaultdict
from torchvision.transforms.functional import to_grayscale
import ast
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from math import ceil 
from matplotlib.ticker import AutoMinorLocator

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!pip install grad-cam

import random
from collections import defaultdict

from pytorch_grad_cam import GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

import torch.nn.functional as F
import cv2


# Specs

In [ ]:






# Get a list of all the files in the directory
file_list = os.listdir(image_dir)

# Calculate the number of elements to remove
num_elements_to_remove = int(len(file_list) * 0.9)

# Randomly select and remove elements
random.shuffle(file_list)
removed_elements = file_list[:num_elements_to_remove]
file_list = file_list[num_elements_to_remove:]


print(len(file_list))
# Find the first image in the list
image_path = None
for filename in file_list:
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_dir, filename)
        break

image = Image.open(image_path)

# Get the size of the image
width, height = image.size
image_input_size = (width,height)
print(f"Image size: {width} x {height}")



#image_dir = 'drive/MyDrive/68300/MLEKG/figures/'



print(image_input_size)

# Y Labels

In [ ]:
Y = pd.read_csv(labels_csv, index_col='ecg_id')
matches = file_list
matches = [int(f.rstrip(image_name_suffix)) for f in matches]
#matches = [21817]

#Remove all rows from Y unless ecg_id is in matches:
Y = Y[Y.index.isin(matches)]

Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(scp_statements_csv, index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_individual_list'] = Y.scp_codes.apply(aggregate_diagnostic)

#BLINKDLY COMMETING TO TAKE JUST THE FIRST ELEMENT!
#Y['diagnostic_superclass'] = Y['diagnostic_superclass'].str[0]

#Keep only the first entry of each list in 'diagnostic_superclass': 


def compare_to_target_list(diagnostic_dict):
    #target_list = ['WPW','2AVB']
    target_list = [
        'NDT', 'NST_', 'DIG', 'LNGQT', 'NORM', 'IMI', 'ASMI', 'LVH', 'LAFB', 'ISC_', 'IRBBB', '1AVB', 'IVCD', 'ISCAL', 'CRBBB',
        'CLBBB', 'ILMI', 'LAO/LAE', 'AMI', 'ALMI', 'ISCIN', 'INJAS', 'LMI', 'ISCIL', 'LPFB', 'ISCAS', 'INJAL', 'ISCLA', 'RVH',
        'ANEUR', 'RAO/RAE', 'EL', 'WPW', 'ILBBB', 'IPLMI', 'MI', 'STTC', 'HYP', 'ISCAN', 'IPMI', 'SEHYP', 'INJIN', 'INJLA',
        'PMI', '3AVB', 'CD', 'INJIL', '2AVB','TRIGU',
    ]
    return_list = [diag for diag in diagnostic_dict.keys() if diag in target_list]
    if len(return_list)<1:
      #print(diagnostic_list)
      return "OTHER"
    if 'TRIGU' in return_list:
      print("RETURNING TRIGU!")
      return 'TRIGU'
    return return_list[0]



print(Y['diagnostic_individual_list'])
# apply function to the dataframe
#Y['diagnostic_superclass'] = Y['diagnostic_individual_list'].apply(map_to_class)
Y['diagnostic_superclass'] = Y['scp_codes'].apply(compare_to_target_list)

print(Y['diagnostic_superclass'])
# for index, row in Y.iterrows():
#     print(f"Row {index} --> diagnostic_individual_list: {row['diagnostic_individual_list']}, diagnostic_superclass: {row['diagnostic_superclass']}")


#classes_to_include = ['NORM', 'STTC']
# # Filter the rows

Y = Y[Y['diagnostic_superclass'].isin(classes_to_include)]




# Transform 'diagnostic_superclass' labels into numerical labels
le = LabelEncoder()
Y['diagnostic_superclass_num'] = le.fit_transform(Y['diagnostic_superclass'])


# Assume that Y['diagnostic_superclass_num'] contains the numerical labels
original_labels = le.inverse_transform(Y['diagnostic_superclass_num'].values)


#print(original_labels)
test_fold = [10]
val_fold = [9]

#y_numerical = Y.diagnostic_superclass.factorize()

unique_labels, label_counts = np.unique(Y.diagnostic_superclass, return_counts=True)

# Print the count of each unique label
total_labels = 0
for label, count in zip(unique_labels, label_counts):
    print(f"Label {label} appears {count} times")
    total_labels = total_labels + count

print(f"All Labels appears {total_labels} times")


y_val = Y[(Y.strat_fold.isin(val_fold))].diagnostic_superclass_num
y_test = Y[Y.strat_fold.isin(test_fold)].diagnostic_superclass_num
y_train = Y[(~Y.strat_fold.isin(val_fold)) & (~Y.strat_fold.isin(test_fold))].diagnostic_superclass_num

In [ ]:


# For reproducibility
torch.manual_seed(1234)
  
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")
    print("You may want to try to use the GPU in Google Colab by clicking in:")
    print("Runtime > Change Runtime type > Hardware accelerator > GPU.")
    
    
def initialize_model(model_name, num_classes, resume_from = None, use_pretrained = False):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    # The model (nn.Module) to return
    model_ft = None
    # The input image is expected to be (input_size, input_size)
    #input_size = 0
    
    # By default, all parameters will be trained (useful when you're starting from scratch)
    # Within this function you can set .requires_grad = False for various parameters, if you
    # don't want to learn them



    if model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = image_input_size


    else:
        raise Exception("Invalid model name!")
    
    if resume_from is not None:
        print("Loading weights from %s" % resume_from)
        model_ft.load_state_dict(torch.load(resume_from))
    
    return model_ft, input_size

def make_optimizer(model, learning_rate, print_parameters=False):
    # Get all the parameters
    params_to_update = model.parameters()
    if print_parameters:
      print("Params to learn:")
      for name, param in model.named_parameters():
          if param.requires_grad == True:
              print("\t",name)

 
    #optimizer = optim.SGD(params_to_update, lr=learning_rate, momentum=0.9)
    optimizer = optim.Adam(params_to_update, lr=learning_rate)
    #optimizer = optim.Adagrad(params_to_update, lr=learning_rate)
    return optimizer

def get_loss():
    # Create an instance of the loss function
    criterion = nn.CrossEntropyLoss()
    return criterion


def train_model(model, dataloaders, criterion, optimizer, save_dir = None, save_all_epochs=False, num_epochs=25):
    '''
    model: The NN to train
    dataloaders: A dictionary containing at least the keys 
                 'train','val' that maps to Pytorch data loaders for the dataset
    criterion: The Loss function
    optimizer: The algorithm to update weights 
               (Variations on gradient descent)
    num_epochs: How many epochs to train for
    save_dir: Where to save the best model weights that are found, 
              as they are found. Will save to save_dir/weights_best.pt
              Using None will not write anything to disk
    save_all_epochs: Whether to save weights for ALL epochs, not just the best
                     validation error epoch. Will save to save_dir/weights_e{#}.pt
    '''
    since = time.time()

    val_acc_history = []
    train_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                print("switching model to train")
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # TQDM has nice progress bars
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    # torch.max outputs the maximum value, and its index
                    # Since the input is batched, we take the max along axis 1
                    # (the meaningful outputs)
                    _, preds = torch.max(outputs, 1)

                    # backprop + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train':
                train_acc_history.append(epoch_acc)
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            if save_all_epochs:
                torch.save(model.state_dict(), os.path.join(save_dir, f'weights_{epoch}.pt'))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # save and load best model weights
    torch.save(best_model_wts, os.path.join(save_dir, 'weights_best_val_acc.pt'))
    torch.save(model.state_dict(), os.path.join(save_dir, 'weights_last.pt'.format(epoch)))
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history

def get_image_transforms():
    # How to transform the image when you are loading them.
    # you'll likely want to mess with the transforms on the training set.
    
    # We convert the image to a [C,H,W] tensor, then normalize it to values with a given mean/stdev. These normalization constants
    # are derived from the mean/stdev of the ImageNet training set which was used to pretrain our models.

    transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    return transform

class CustomDataset(Dataset):
    def __init__(self, data_dir, image_ids, labels, transform=None):
        self.data_dir = data_dir
        self.image_ids = image_ids
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, f"{str(self.image_ids[idx]).zfill(5)}{image_name_suffix}")
        image = Image.open(img_name)
        
        # Check if the image has a corresponding label
        if self.labels is not None:
            label = self.labels[idx]
        else:
            label = None
        
        # Apply data augmentation if provided
        if self.transform:
            image = self.transform(image)

        return image, label


def make_balanced_sampler(labels):
    class_counts = np.bincount(labels)
    class_weights = 1. / class_counts
    samples_weights = class_weights[labels]
    sampler = WeightedRandomSampler(weights=samples_weights, num_samples=len(samples_weights), replacement=True)
    return sampler


def get_dataloaders(dataset_dir, input_size, batch_size, shuffle=True, transform=get_image_transforms()):
    data_transforms = {
        'train': transform,
        'val': transform,
        'test': transform
    }

    # Get the list of image IDs with labels
    train_image_ids = y_train.index[y_train.notnull()]
    val_image_ids = y_val.index[y_val.notnull()]
    test_image_ids = y_test.index[y_test.notnull()]

    # Get the labels for the images with labels
    train_labels = y_train.loc[train_image_ids].to_numpy()
    val_labels = y_val.loc[val_image_ids].to_numpy()
    test_labels = y_test.loc[test_image_ids].to_numpy()
    
    # Make a balanced sampler for the training set
    train_sampler = make_balanced_sampler(train_labels)

    # Create the datasets
    train_dataset = CustomDataset(dataset_dir, train_image_ids, train_labels, transform=data_transforms['train'])
    val_dataset = CustomDataset(dataset_dir, val_image_ids, val_labels, transform=data_transforms['val'])
    test_dataset = CustomDataset(dataset_dir, test_image_ids, test_labels, transform=data_transforms['test'])

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=4)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4)

    dataloaders_dict = {
        'train': train_dataloader,
        'val': val_dataloader,
        'test': test_dataloader
    }
    return dataloaders_dict


# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
# You can add your own, or modify these however you wish!

# Number of classes in the dataset
num_classes = Y['diagnostic_superclass'].nunique()

# Batch size for training (change depending on how much memory you have)

# Shuffle the input data?
shuffle_datasets = True

# Number of epochs to train for 

# Learning rate
learning_rate = 0.001

### IO
# Path to a model file to use to start weights at
resume_from = None

# Whether to use a pretrained model, trained for classification in Imagenet-1k 
pretrained = False

# Save all epochs so that you can select the model from a particular epoch
save_all_epochs = False

# Whether to use early stopping (load the model with best accuracy), or not
early_stopping = True

# Directory to save weights to
os.makedirs(save_dir, exist_ok=True)

# Initialize the model for this run
# train model_1
model_1, input_size = initialize_model(model_name = model_name, num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)
dataloaders = get_dataloaders(image_dir, input_size, batch_size, shuffle_datasets)
criterion = get_loss()


train_dataloader = dataloaders['train']

num_batches_to_print = 3




# Training

In [ ]:
print(unique_labels, label_counts)

In [ ]:
if train_the_model:
  # Move the model to the gpu if needed
  model_1 = model_1.to(device)

  optimizer_1 = make_optimizer(model_1, learning_rate)

  # Train the model!
  trained_model_1, validation_history_1, train_history_1 = train_model(model=model_1, 
                                                                      dataloaders=dataloaders, 
                                                                      criterion=criterion, 
                                                                      optimizer=optimizer_1,
                                                                      save_dir=save_dir, 
                                                                      save_all_epochs=save_all_epochs, 
                                                                      num_epochs=num_epochs)
  del model_1, optimizer_1, trained_model_1


# Final

In [ ]:
# Load your final model, that we will use for the rest of the PSET.
if early_stopping:
  weights_file = save_dir + '/weights_best_val_acc.pt'
else:
  weights_file = save_dir + '/weights_last.pt'
model_yours, _ = initialize_model(model_name = model_name, num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)

# Move the model to the gpu if needed
model_yours = model_yours.to(device)

# Load weights for model_yours
#model_yours.load_state_dict(torch.load(weights_file))
model_yours.load_state_dict(torch.load(weights_file,map_location=torch.device('cpu')))


# set models to eval mode
model_yours = model_yours.eval()


# Confusion Mat

In [ ]:
def evaluate(model, dataloader, criterion, is_labelled = False, generate_labels = True, k = 5):
    # If is_labelled, we want to compute loss, top-1 accuracy and top-5 accuracy
    # If generate_labels, we want to output the actual labels
    # Set the model to evaluate mode
    model.eval()
    running_loss = 0
    running_top1_correct = 0
    running_top5_correct = 0
    predicted_labels = []
    gt_labels = []

    # Iterate over data.
    # TQDM has nice progress bars
    for inputs, labels in tqdm(dataloader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        tiled_labels = torch.stack([labels.data for i in range(k)], dim=1) 
        # Makes this to calculate "top 5 prediction is correct"
        # [[label1 label1 label1 label1 label1], [label2 label2 label2 label label2]]

        # forward
        # track history if only in train
        with torch.set_grad_enabled(False):
            # Get model outputs and calculate loss
            outputs = model(inputs)
            if is_labelled:
                loss = criterion(outputs, labels)

            # torch.topk outputs the maximum values, and their indices
            # Since the input is batched, we take the max along axis 1
            # (the meaningful outputs)
            _, preds = torch.topk(outputs, k=k, dim=1)
            if generate_labels:
                # We want to store these results
                nparr = preds.cpu().detach().numpy()
                predicted_labels.extend([list(nparr[i]) for i in range(len(nparr))])
                gt_labels.extend(np.array(labels.cpu()))

        if is_labelled:
            # statistics
            running_loss += loss.item() * inputs.size(0)
            # Check only the first prediction
            running_top1_correct += torch.sum(preds[:, 0] == labels.data)
            # Check all 5 predictions
            running_top5_correct += torch.sum(preds == tiled_labels)
        else:
            pass

    # Only compute loss & accuracy if we have the labels
    if is_labelled:
        epoch_loss = float(running_loss / len(dataloader.dataset))
        epoch_top1_acc = float(running_top1_correct.double() / len(dataloader.dataset))
        epoch_top5_acc = float(running_top5_correct.double() / len(dataloader.dataset))
    else:
        epoch_loss = None
        epoch_top1_acc = None
        epoch_top5_acc = None
    
    # Return everything
    return epoch_loss, epoch_top1_acc, gt_labels, predicted_labels  

# Wrapper to easily evaulate a model given a model and the set of dataloaders
def get_eval_results(model, dataloaders):
    model.eval()
    true_label_list = []
    outputs_list = []
    predicted_label_list = []
    original_image_list = []

    # TQDM has nice progress bars
    for inputs, labels in tqdm(dataloaders['test']):
        inputs = inputs.to(device)
        labels = labels.to(device)
        with torch.set_grad_enabled(False):
            # Get model outputs and calculate loss
            outputs = model(inputs)
            true_label_list.append(labels)
            original_image_list.append(inputs)
            outputs_list.append(outputs)
            _, preds = torch.topk(outputs, k=1, dim=1)
            predicted_label_list.append(preds)
    return torch.concat(true_label_list).unsqueeze(-1).cpu().numpy(), \
           torch.concat(predicted_label_list).cpu().numpy(), \
           torch.softmax(torch.concat(outputs_list), dim=1).cpu().numpy(), \
           torch.concat(original_image_list).cpu().numpy()


# Get data on the validation set
# Setting this to false will be a little bit faster
generate_validation_labels = True

model = model_yours

## Please make sure you understand what outputs means here
y_label, y_pred, outputs, inputs =  get_eval_results(model, dataloaders)

In [ ]:
print(y_label)
print(label_map)

In [ ]:
def plot_confusion_matrix(y_label, y_pred, title='Confusion matrix'):
    # Calculate the confusion matrix
    num_classes = len(np.unique(y_label))
    print(num_classes)
    confusion_matrix = np.zeros([num_classes,num_classes])
    for i in range(len(y_label)):
        confusion_matrix[y_label[i], y_pred[i]] += 1

    # Save raw numbers for display
    confusion_matrix_raw = confusion_matrix 

    # Convert to percentage
    confusion_matrix = confusion_matrix / confusion_matrix.sum(axis=1)[:, np.newaxis]

    # Plot the confusion matrix as a heatmap with sorted vertical axis
    fig, ax = plt.subplots()

    # Annoying loops for labeling
    for i in np.arange(0,num_classes):
        for j in np.arange(0,num_classes):
            #ax.text(j + 0.5, i + 0.5, '{:.1%}'.format(confusion_matrix[i, j]),
            #       ha='center', va='center', color='black')
            ax.text(j + 0.5, i + 0.5, '{:.1%}\n{:.0f}'.format(confusion_matrix[i, j], confusion_matrix_raw[i, j]),
                    ha='center', va='center', color='black')


    sns.heatmap(confusion_matrix, cmap='Blues', annot=False)
   
    ax.set_xticklabels(classes, rotation=90)  # Set text labels on x axis
    ax.set_yticklabels(classes, rotation=0)   # Set text labels on y axis

    ax.set_xlabel('Predicted')  # Set x-axis label
    ax.set_ylabel('Truth')  # Set y-axis label

    ax.set_title(title)
    fig.tight_layout()
    plt.show()
    return confusion_matrix
    
classes = le.classes_

label_map = {i: label for i, label in enumerate(classes)}

confusion_matrix = plot_confusion_matrix(y_label, y_pred)
print(confusion_matrix)

In [ ]:
def plot_confusion_matrix(y_label, y_pred, title='Confusion matrix'):
    # Calculate the confusion matrix
    num_classes = len(np.unique(y_label))
    print(num_classes)
    confusion_matrix = np.zeros([num_classes,num_classes])
    for i in range(len(y_label)):
        confusion_matrix[y_label[i], y_pred[i]] += 1

    # Save raw numbers for display
    confusion_matrix_raw = confusion_matrix 

    # Convert to percentage
    confusion_matrix = confusion_matrix / confusion_matrix.sum(axis=1)[:, np.newaxis]

    # Plot the confusion matrix as a heatmap with sorted vertical axis
    fig, ax = plt.subplots()

    # Annoying loops for labeling
    for i in np.arange(0,num_classes):
        for j in np.arange(0,num_classes):
            #ax.text(j + 0.5, i + 0.5, '{:.1%}'.format(confusion_matrix[i, j]),
            #       ha='center', va='center', color='black')
            ax.text(j + 0.5, i + 0.5, '{:.1%}\n{:.0f}'.format(confusion_matrix[i, j], confusion_matrix_raw[i, j]),
                    ha='center', va='center', color='black')


    sns.heatmap(confusion_matrix, cmap='Blues', annot=False)
   
    ax.set_xticklabels(classes, rotation=90)  # Set text labels on x axis
    ax.set_yticklabels(classes, rotation=0)   # Set text labels on y axis

    ax.set_xlabel('Predicted')  # Set x-axis label
    ax.set_ylabel('Truth')  # Set y-axis label

    ax.set_title(title)
    fig.tight_layout()
    plt.show()
    return confusion_matrix
    
classes = le.classes_

label_map = {i: label for i, label in enumerate(classes)}

confusion_matrix = plot_confusion_matrix(y_label, y_pred)

In [ ]:
import numpy as np

def calculate_sensitivity_specificity(confusion_matrix):
    # Ensure the confusion matrix is a numpy array
    confusion_matrix = np.array(confusion_matrix)

    # Calculate the total sum of the confusion matrix
    total = np.sum(confusion_matrix)
    
    if confusion_matrix.shape[0] != confusion_matrix.shape[1]:
        return "The provided confusion matrix is not square"
    
    sensitivity = []
    specificity = []
    
    for i in range(confusion_matrix.shape[0]):
        # Sensitivity (also known as true positive rate or recall)
        tp = confusion_matrix[i,i]
        fn = np.sum(confusion_matrix[i,:]) - tp
        sensitivity.append(tp / (tp + fn))

        # Specificity (also known as true negative rate)
        tn = total - np.sum(confusion_matrix[i,:]) - np.sum(confusion_matrix[:,i]) + tp
        fp = np.sum(confusion_matrix[:,i]) - tp
        specificity.append(tn / (tn + fp))

    return sensitivity, specificity

sensitivity, specificity = calculate_sensitivity_specificity(confusion_matrix)
print(label_map)
print("Sensitivity: ", sensitivity)
print("Specificity: ", specificity)

# Assuming the label_map, sensitivity, and specificity are defined
for i in range(len(label_map)):
    print(f"{label_map[i]} - Sensitivity: {sensitivity[i]:.4f}, Specificity: {specificity[i]:.4f}")


# {0: 'CD', 1: 'HYP', 2: 'MI', 3: 'NORM', 4: 'STTC'}
# Sensitivity:  [0.06451612903225806, 0.0, 0.47368421052631576, 0.9770114942528736, 0.7647058823529411]
# Specificity:  [1.0, 1.0, 0.8899430740037951, 0.5959419421418823, 0.8340944128954199]




In [ ]:

class GradCAM:
    def __init__(self, model):
        self.model = model
        self.feature = None
        self.gradient = None

    def save_gradient(self, grad):
        self.gradient = grad

    def __call__(self, x):
        image_size = (x.size(-1), x.size(-2))
        feature_maps = []

        for i in range(x.size(0)):
            img = x[i].data.cpu().numpy()
            img = img - np.min(img)
            if np.max(img) != 0:
                img = img / np.max(img)

            feature = x[i].unsqueeze(0)

            for name, module in self.model.named_children():
                if name == 'classifier':
                    feature = feature.view(feature.size(0), -1)
                feature = module(feature)
                if name == 'features':
                    feature.register_hook(self.save_gradient)
                    self.feature = feature
            classes = F.sigmoid(feature)
            one_hot, _ = classes.max(dim=-1)
            self.model.zero_grad()
            one_hot.backward()
            weight = self.gradient.mean(dim=-1, keepdim=True).mean(dim=-2, keepdim=True)
            mask = F.relu((weight * self.feature).sum(dim=1)).squeeze(0)
            mask = cv2.resize(mask.data.cpu().numpy(), image_size)
            mask = mask - np.min(mask)

            if np.max(mask) != 0:
                mask = mask / np.max(mask)

            feature_map = np.float32(cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET))
            cam = feature_map + np.float32((np.uint8(img.transpose((1, 2, 0)) * 255)))
            cam = cam - np.min(cam)

            if np.max(cam) != 0:
                cam = cam / np.max(cam)

            feature_maps.append(transforms.ToTensor()(cv2.cvtColor(np.uint8(255 * cam), cv2.COLOR_BGR2RGB)))

        feature_maps = torch.stack(feature_maps)

        return feature_maps

# def visualize(img, cam):
#     plt.figure(figsize=(10,10))
#     plt.subplot(1,2,1)
#     plt.imshow(np.transpose(img,(1,2,0)))
#     plt.title('Input Image')
#     plt.subplot(1,2,2)
#     plt.imshow(np.transpose(cam,(1,2,0)))
#     plt.title('Grad-CAM')
#     plt.show()

# def visualize(img, cam, label):
#     plt.figure(figsize=(10,10))
#     plt.subplot(1,2,1)
#     plt.imshow(np.transpose(img,(1,2,0)))
#     if label == 0:
#       label = "SBRAD"
#     elif label == 1:
#       label = "AFIB"
#     plt.title('Input Image - Label: {}'.format(label))
#     plt.subplot(1,2,2)
#     plt.imshow(np.transpose(cam,(1,2,0)))
#     plt.title('Grad-CAM - Label: {}'.format(label))
#     plt.show()

def visualize(img, cam, ground_truth, prediction):
    plt.figure(figsize=(10,10))
    plt.subplot(1,2,1)
    plt.imshow(np.transpose(img,(1,2,0)))
    plt.title('Input Image - Ground Truth: {}'.format(ground_truth))
    plt.subplot(1,2,2)
    plt.imshow(np.transpose(cam,(1,2,0)))
    plt.title('Grad-CAM - Predicted: {}'.format(prediction))
    plt.show()



In [ ]:
from torchvision import transforms

original_labels = le.inverse_transform(Y['diagnostic_superclass_num'].values)
label_map = {i: label for i, label in enumerate(original_labels)}

# Assume that dataloaders['test'] and model are already defined
grad_cam = GradCAM(model)

# Loop over all batches in the 'test' set
for i, (inputs, labels) in enumerate(dataloaders['test']):
    inputs = inputs.to(device)
    labels = labels.to(device)
    # Make a prediction for each input
    outputs = model(inputs)
    #test_loss_yours, test_top1_yours, _, test_labels_yours
    #outputs = test_labels_yours
    _, preds = torch.max(outputs, 1)

    # Apply GradCAM
    cams = grad_cam(inputs)

    # Visualize the original images and Grad-CAM heatmaps
    for j in range(inputs.size(0)):
        ground_truth = label_map[labels[j].item()]
        prediction = label_map[preds[j].item()]
        visualize(inputs[j].cpu(), cams[j].cpu(), ground_truth, prediction)

    # You may want to break the loop after a few batches to avoid producing too many images
    if i == 10:  # Just print for 5 batches
        break


# Grad CAM

In [ ]:
def grad_cam(input_model, image, class_idx, feature_module):
    model_features = list(input_model.children())[:-1]

    model = torch.nn.Sequential(*model_features)

    feature = model(image)
    feature = feature.detach().cpu().numpy()

    weight_softmax_params = list(input_model.parameters())[-2]
    weight_softmax = np.squeeze(weight_softmax_params.detach().cpu().numpy())

    class_weights = weight_softmax[class_idx]
    cam = feature.dot(class_weights)

    # Process CAM
    cam = cv2.resize(cam, (224, 224))
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    return cam

def plot_grad_cam(cam, img):
    img = np.transpose(img, (1,2,0))
    img = (img - np.min(img)) / np.ptp(img)
    cam_heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    cam_heatmap = cv2.cvtColor(cam_heatmap, cv2.COLOR_BGR2RGB)
    cam = np.float32(cam_heatmap) + np.float32(img)
    cam = 255 * cam / np.max(cam)

    plt.figure(figsize=(8, 8))
    plt.imshow(np.uint8(cam))
    plt.show()

y_label, y_pred, outputs, inputs = get_eval_results(model, dataloaders)

# Visualize Grad-CAM for correctly and incorrectly predicted images
for i in range(len(y_label)):
    img = inputs[i]
    true_label = y_label[i]
    pred_label = y_pred[i]

    cam = grad_cam(model, img, pred_label, model.layer4)
    plot_grad_cam(cam, img)

    if i > 10: # Limit to 10 images for brevity
        break


In [ ]:

# Create a dictionary that maps the integer label to the alphabetical label

print(label_map)
# Print the mapping
for integer, label in label_map.items():
    print(f"{integer} --> {label}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def precision_recall_curve(probabilities, y_label, y_pred):
    end = len(y_label)
    precisions = []
    recalls = []
    decision_thresholds = np.arange(0,1,0.001)
    
    new_y_label = []
    for i in range(0,end):
      if y_label[i] == 1:
        new_y_label.append(1)
      else:
        new_y_label.append(0)
        

    for decision_threshold in decision_thresholds:
      new_y_pred = []

  
      for i in range(0,end):
        probs = probabilities[i]
        ground_truth = y_label[i]
        if probs[1] > decision_threshold:
          new_y_pred.append(1)
        else:
          new_y_pred.append(0)

      # Calculate true/false positive/negative
      tp,fp,tn,fn = 0,0,0,0
      for i in range(len(new_y_label)):
            if new_y_label[i] == 1 and new_y_pred[i] == 1:
                tp += 1
            elif new_y_label[i] == 0 and new_y_pred[i] == 1:
                fp += 1
            elif new_y_label[i] == 0 and new_y_pred[i] == 0:
                tn += 1
            elif new_y_label[i] == 1 and new_y_pred[i] == 0:
                fn += 1

      # Handle edge cases (not sure if there is a different convention needed)
      if (tp+fn)==0:
        recall = 1
      else:
        recall = tp/(tp+fn)

      if (tp+fp)==0:
        precision= 1
      else:
        precision = tp/(tp+fp)

      precisions.append(precision)
      recalls.append(recall)

    # Finally, calulate AUC
    auc = np.trapz(recalls,precisions)
    annotation="AUC: {:.5}".format(auc)

    #Reference for color line: https://matplotlib.org/stable/gallery/lines_bars_and_markers/multicolored_line.html
    # I couldn't get a color line to work (not neede for the homework, is just cool)
    # But Im leaving this in to hopefully come back to it later.

    # Plot it
    fig, ax = plt.subplots()
    plt.plot(recalls, precisions)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.xlim(0,1.1)
    plt.ylim(0,0.7)
    plt.text(0.1,0.1,annotation, fontsize=12)
    plt.show()

    return(precisions,recalls)

precisions,recalls = precision_recall_curve(outputs, y_label, y_pred)



In [ ]:
def plot_cams(target_dataloaders, cam_method=GradCAM, class_id='gt'):
  #assert class_id in ['gt', 'pred'] or type(class_id) is int and class_id < 3
  for split in ['train', 'val', 'test']:
    #target_layers = [model.layer4[-1]]
    target_layer=model.features[-1]
    fig, axs = plt.subplots(2, 6, figsize=(14, 7))
    fig.suptitle("Examples split {}, class {}".format(split, class_id), fontsize=16)

    split_dataset = target_dataloaders[split].dataset
    indices_per_class = defaultdict(list)
    for i, (_, c) in enumerate(split_dataset.imgs):
      indices_per_class[c].append(i)

    random.seed(1337)  
    indices = []
    for c in range(3):
      indices.extend(random.sample(indices_per_class[c], 4))

    for dataset_i, ax in zip(indices, axs.flatten()):
      input_tensor, class_idx = split_dataset[dataset_i]
      input_tensor = input_tensor[None,...].cuda()

      # Construct the CAM object once, and then re-use it on many images:
      cam = cam_method(model=model, target_layers=target_layers, use_cuda=not device == 'cpu')
      pred_class_idx = model(input_tensor).argmax()

      # We have to specify the target we want to generate
      # the Class Activation Maps for
      if type(class_id) is int:
        target_class_id = class_id
      else:
        target_class_id = pred_class_idx if class_id == 'pred' else class_idx        
      targets = [ClassifierOutputTarget(target_class_id)]

      # You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
      grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

      # In this example grayscale_cam has only one image in the batch:
      grayscale_cam = grayscale_cam[0, :]
      rgb_image = np.array(input_tensor[0].cpu())
      rgb_image = (rgb_image - rgb_image.min())/ (rgb_image.max() - rgb_image.min())
      visualization = show_cam_on_image(rgb_image.transpose((1,2,0)), grayscale_cam, use_rgb=True)
      
      ax.set_title('True : %s\n Predicted: %s' %(split_dataset.classes[class_idx], split_dataset.classes[pred_class_idx]))
      ax.imshow(visualization)


In [ ]:
# Substitute GradCAM for one of the following, to test other methods:
# GradCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
# Also substitute class_id for pred, or a class id in [0,1,2] to check different output.
#plot_cams(dataloaders, cam_method=GradCAM, class_id='gt')
#plot_cams(dataloaders, cam_method=GradCAM, class_id='pred')
#plot_cams(dataloaders, cam_method=GradCAM, class_id=1)
plot_cams(dataloaders, cam_method=GradCAM, class_id=3)



In [ ]:
import random
from collections import defaultdict

class RebalancedDataset(datasets.ImageFolder):
  def __init__(self, class_probabilities, *args, **kwargs):
    super().__init__(*args, **kwargs)
    self.class_probabilities = class_probabilities
    self.items_per_class = defaultdict(list)
    
    for item_i, (_, c) in enumerate(self.imgs):
      self.items_per_class[c].append(item_i)

  def __getitem__(self, i_ignored):
    # ignores argument, and returns an element at random with reweighted class
    if i_ignored >= len(self):
      raise IndexError('Index is too large') # Exception required so that the iterator ends after len(self) samples

    # TODO: select a class at random following self.class_probabilities, and then select a random element for the class
    class_i = random.choices(range(len(self.classes)), weights=self.class_probabilities)[0]
    i = random.choice(self.items_per_class[class_i])
    # ENDTODO
    return super().__getitem__(i)


In [ ]:
# Replicates the original get_dataloaders, but in this case we substitute the training dataset for the RebalancedDataset
def get_dataloaders_rebalanced(class_probabilities, dataset_dir, input_size, batch_size, shuffle = True):
    data_transforms = {
        'train': get_image_transforms(),
        'val': get_image_transforms(),
        'test': get_image_transforms()
    }

    # Create training and validation datasets
    image_datasets = {'train': RebalancedDataset(class_probabilities, os.path.join(dataset_dir, 'train'), data_transforms['train']),
                      'val': datasets.ImageFolder(os.path.join(dataset_dir, 'val'), data_transforms['val']),
                      'test': datasets.ImageFolder(os.path.join(dataset_dir, 'test'), data_transforms['test'])}
    # Create training and validation dataloaders
    # Never shuffle the test set
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=False if x != 'train' else shuffle, num_workers=4) for x in data_transforms.keys()}
    return dataloaders_dict

In [ ]:
class_probabilities = [1/3 for _ in range(3)]
dataloaders_rebalanced = get_dataloaders_rebalanced(class_probabilities, mias_dataset_dir, input_size, batch_size, shuffle_datasets)

train_dataset_rebalanced = dataloaders_rebalanced['train'].dataset


In [ ]:
from collections import defaultdict 

samples_per_class_rebalanced = [0 for _ in range(3)]
for _, class_idx in tqdm(train_dataset_rebalanced):
  samples_per_class_rebalanced[class_idx] += 1

print("Rebalanced samples per class:")
for c_idx in range(3):
  print("Class {} ({}): {} samples".format(c_idx, train_dataset_rebalanced.classes[c_idx], samples_per_class_rebalanced[c_idx]))



In [ ]:
model_rebalanced, input_size = initialize_model(model_name = model_name, num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)

model_rebalanced = model_rebalanced.to(device)
optimizer_rebalanced = make_optimizer(model_rebalanced, learning_rate)
trained_rebalanced, validation_history_rebalanced, train_history_rebalanced = train_model(model=model_rebalanced, 
                                                                                          dataloaders=dataloaders_rebalanced, 
                                                                                          criterion=criterion, 
                                                                                          optimizer=optimizer_rebalanced,
                                                                                          save_dir=save_dir, 
                                                                                          save_all_epochs=save_all_epochs, 
                                                                                          num_epochs=num_epochs)

In [ ]:
y_label_rebalanced, y_pred_rebalanced, _, _ =  get_eval_results(model_rebalanced, dataloaders)
plot_confusion_matrix(y_label_rebalanced, y_pred_rebalanced, title='CM rebalanced')

In [ ]:
drive.mount('/content/drive')
save_dir = 'drive/MyDrive/68300/MLEKG/trained_models/{}'.format(model_label)
classes_to_include = ['NORM','1AVB','WPW','TRIGU']


# Get a list of all the files in the directory
file_list = os.listdir(image_dir)

print(len(file_list))
# Find the first image in the list
image_path = None
for filename in file_list:
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_dir, filename)
        break

image = Image.open(image_path)

# Get the size of the image
width, height = image.size
image_input_size = (width,height)
print(f"Image size: {width} x {height}")



#image_dir = 'drive/MyDrive/68300/MLEKG/figures/'

print(image_input_size)

Y = pd.read_csv(labels_csv, index_col='ecg_id')
matches = os.listdir(image_dir)
matches = [int(f.rstrip(image_name_suffix)) for f in matches]
#matches = [21817]

#Remove all rows from Y unless ecg_id is in matches:
Y = Y[Y.index.isin(matches)]

Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(scp_statements_csv, index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_individual_list'] = Y.scp_codes.apply(aggregate_diagnostic)

def compare_to_target_list(diagnostic_dict):
    #target_list = ['WPW','2AVB']
    target_list = [
        'NDT', 'NST_', 'DIG', 'LNGQT', 'NORM', 'IMI', 'ASMI', 'LVH', 'LAFB', 'ISC_', 'IRBBB', '1AVB', 'IVCD', 'ISCAL', 'CRBBB',
        'CLBBB', 'ILMI', 'LAO/LAE', 'AMI', 'ALMI', 'ISCIN', 'INJAS', 'LMI', 'ISCIL', 'LPFB', 'ISCAS', 'INJAL', 'ISCLA', 'RVH',
        'ANEUR', 'RAO/RAE', 'EL', 'WPW', 'ILBBB', 'IPLMI', 'MI', 'STTC', 'HYP', 'ISCAN', 'IPMI', 'SEHYP', 'INJIN', 'INJLA',
        'PMI', '3AVB', 'CD', 'INJIL', '2AVB','TRIGU',
    ]
    return_list = [diag for diag in diagnostic_dict.keys() if diag in target_list]
    if len(return_list)<1:
      #print(diagnostic_list)
      return "OTHER"
    if 'TRIGU' in return_list:
      print("RETURNING TRIGU!")
      return 'TRIGU'
    return return_list[0]



print(Y['diagnostic_individual_list'])
# apply function to the dataframe
#Y['diagnostic_superclass'] = Y['diagnostic_individual_list'].apply(map_to_class)
Y['diagnostic_superclass'] = Y['scp_codes'].apply(compare_to_target_list)

print(Y['diagnostic_superclass'])
# for index, row in Y.iterrows():
#     print(f"Row {index} --> diagnostic_individual_list: {row['diagnostic_individual_list']}, diagnostic_superclass: {row['diagnostic_superclass']}")


#classes_to_include = ['NORM', 'STTC']
# # Filter the rows

Y = Y[Y['diagnostic_superclass'].isin(classes_to_include)]




# Transform 'diagnostic_superclass' labels into numerical labels
le = LabelEncoder()
Y['diagnostic_superclass_num'] = le.fit_transform(Y['diagnostic_superclass'])


# Assume that Y['diagnostic_superclass_num'] contains the numerical labels
original_labels = le.inverse_transform(Y['diagnostic_superclass_num'].values)


#print(original_labels)
test_fold = [10]
val_fold = [9]

#y_numerical = Y.diagnostic_superclass.factorize()

unique_labels, label_counts = np.unique(Y.diagnostic_superclass, return_counts=True)

# Print the count of each unique label
total_labels = 0
for label, count in zip(unique_labels, label_counts):
    print(f"Label {label} appears {count} times")
    total_labels = total_labels + count

print(f"All Labels appears {total_labels} times")


y_val = Y[(Y.strat_fold.isin(val_fold))].diagnostic_superclass_num
y_test = Y[Y.strat_fold.isin(test_fold)].diagnostic_superclass_num
y_train = Y[(~Y.strat_fold.isin(val_fold)) & (~Y.strat_fold.isin(test_fold))].diagnostic_superclass_num



# For reproducibility
torch.manual_seed(1234)
  
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("Using the GPU!")
else:
    print("WARNING: Could not find GPU! Using CPU only")
    print("You may want to try to use the GPU in Google Colab by clicking in:")
    print("Runtime > Change Runtime type > Hardware accelerator > GPU.")
    
    
def initialize_model(model_name, num_classes, resume_from = None, use_pretrained = False):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    # The model (nn.Module) to return
    model_ft = None
    # The input image is expected to be (input_size, input_size)
    #input_size = 0
    
    # By default, all parameters will be trained (useful when you're starting from scratch)
    # Within this function you can set .requires_grad = False for various parameters, if you
    # don't want to learn them

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = image_input_size
        

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = image_input_size

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = image_input_size



    else:
        raise Exception("Invalid model name!")
    
    if resume_from is not None:
        print("Loading weights from %s" % resume_from)
        model_ft.load_state_dict(torch.load(resume_from))
    
    return model_ft, input_size

def make_optimizer(model, learning_rate, print_parameters=False):
    # Get all the parameters
    params_to_update = model.parameters()
    if print_parameters:
      print("Params to learn:")
      for name, param in model.named_parameters():
          if param.requires_grad == True:
              print("\t",name)

 
    #optimizer = optim.SGD(params_to_update, lr=learning_rate, momentum=0.9)
    optimizer = optim.Adam(params_to_update, lr=learning_rate)
    #optimizer = optim.Adagrad(params_to_update, lr=learning_rate)
    return optimizer

def get_loss():
    # Create an instance of the loss function
    criterion = nn.CrossEntropyLoss()
    return criterion


def train_model(model, dataloaders, criterion, optimizer, save_dir = None, save_all_epochs=False, num_epochs=25):
    '''
    model: The NN to train
    dataloaders: A dictionary containing at least the keys 
                 'train','val' that maps to Pytorch data loaders for the dataset
    criterion: The Loss function
    optimizer: The algorithm to update weights 
               (Variations on gradient descent)
    num_epochs: How many epochs to train for
    save_dir: Where to save the best model weights that are found, 
              as they are found. Will save to save_dir/weights_best.pt
              Using None will not write anything to disk
    save_all_epochs: Whether to save weights for ALL epochs, not just the best
                     validation error epoch. Will save to save_dir/weights_e{#}.pt
    '''
    since = time.time()

    val_acc_history = []
    train_acc_history = []
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                print("switching model to train")
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # TQDM has nice progress bars
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)


                    _, preds = torch.max(outputs, 1)

                    # backprop + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'train':
                train_acc_history.append(epoch_acc)
            if phase == 'val':
                val_acc_history.append(epoch_acc)
            if save_all_epochs:
                torch.save(model.state_dict(), os.path.join(save_dir, f'weights_{epoch}.pt'))

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # save and load best model weights
    torch.save(best_model_wts, os.path.join(save_dir, 'weights_best_val_acc.pt'))
    torch.save(model.state_dict(), os.path.join(save_dir, 'weights_last.pt'.format(epoch)))
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history

def get_image_transforms():

    transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    return transform

class CustomDataset(Dataset):
    def __init__(self, data_dir, image_ids, labels, transform=None):
        self.data_dir = data_dir
        self.image_ids = image_ids
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, f"{str(self.image_ids[idx]).zfill(5)}{image_name_suffix}")
        image = Image.open(img_name)
        
        # Check if the image has a corresponding label
        if self.labels is not None:
            label = self.labels[idx]
        else:
            label = None
        
        # Apply data augmentation if provided
        if self.transform:
            image = self.transform(image)

        return image, label


from torch.utils.data import WeightedRandomSampler

def make_balanced_sampler(labels):
    class_counts = np.bincount(labels)
    class_weights = 1. / class_counts
    samples_weights = class_weights[labels]
    sampler = WeightedRandomSampler(weights=samples_weights, num_samples=len(samples_weights), replacement=True)
    return sampler


def get_dataloaders(dataset_dir, input_size, batch_size, shuffle=True, transform=get_image_transforms()):
    data_transforms = {
        'train': transform,
        'val': transform,
        'test': transform
    }

    # Get the list of image IDs with labels
    train_image_ids = y_train.index[y_train.notnull()]
    val_image_ids = y_val.index[y_val.notnull()]
    test_image_ids = y_test.index[y_test.notnull()]

    # Get the labels for the images with labels
    train_labels = y_train.loc[train_image_ids].to_numpy()
    val_labels = y_val.loc[val_image_ids].to_numpy()
    test_labels = y_test.loc[test_image_ids].to_numpy()
    
    # Make a balanced sampler for the training set
    train_sampler = make_balanced_sampler(train_labels)

    # Create the datasets
    train_dataset = CustomDataset(dataset_dir, train_image_ids, train_labels, transform=data_transforms['train'])
    val_dataset = CustomDataset(dataset_dir, val_image_ids, val_labels, transform=data_transforms['val'])
    test_dataset = CustomDataset(dataset_dir, test_image_ids, test_labels, transform=data_transforms['test'])

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=4)
    val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4)

    dataloaders_dict = {
        'train': train_dataloader,
        'val': val_dataloader,
        'test': test_dataloader
    }
    return dataloaders_dict



# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet]
# You can add your own, or modify these however you wish!

# Number of classes in the dataset
num_classes = Y['diagnostic_superclass'].nunique()

# Batch size for training (change depending on how much memory you have)

# Shuffle the input data?
shuffle_datasets = True

# Number of epochs to train for 

# Learning rate
learning_rate = 0.001

### IO
# Path to a model file to use to start weights at
resume_from = None

# Whether to use a pretrained model, trained for classification in Imagenet-1k 
pretrained = False

# Save all epochs so that you can select the model from a particular epoch
save_all_epochs = False

# Whether to use early stopping (load the model with best accuracy), or not
early_stopping = True

# Directory to save weights to
os.makedirs(save_dir, exist_ok=True)

# Initialize the model for this run
# train model_1
model_1, input_size = initialize_model(model_name = model_name, num_classes = num_classes, resume_from=resume_from, use_pretrained=pretrained)
dataloaders = get_dataloaders(image_dir, input_size, batch_size, shuffle_datasets)
criterion = get_loss()


train_dataloader = dataloaders['train']

num_batches_to_print = 3

if train_the_model:
  # Move the model to the gpu if needed
  model_1 = model_1.to(device)

  optimizer_1 = make_optimizer(model_1, learning_rate)

  # Train the model!
  trained_model_1, validation_history_1, train_history_1 = train_model(model=model_1, 
                                                                      dataloaders=dataloaders, 
                                                                      criterion=criterion, 
                                                                      optimizer=optimizer_1,
                                                                      save_dir=save_dir, 
                                                                      save_all_epochs=save_all_epochs, 
                                                                      num_epochs=num_epochs)
  del model_1, optimizer_1, trained_model_1
